In [ ]:
!pip install transformers==4.45.2 accelerate -q

import torch
from transformers import AutoModelForCausalLM, AutoProcessor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import zipfile
import json
import os
from PIL import Image

model_id = "allenai/Molmo-7B-D-0924"
device = 'cuda'

In [ ]:
# load model
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype="auto", device_map="auto", trust_remote_code=True
)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)



In [ ]:
extract_path = "restored_images"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile("dataset_images.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_path)

with open("dataset.json", 'r') as f:
    loaded_metadata = json.load(f)

dataset = []
for entry in loaded_metadata:
    image_path = os.path.join(extract_path, f"{entry['index']}.jpg")
    image = Image.open(image_path)

    dst_item = {
        "text": entry["label"],
        "image": image,
        "label": entry["count"]
    }
    dataset.append(dst_item)

group_size = 56

dataset = dataset[:group_size*5]

In [ ]:
print(len(dataset), dataset[0])

In [ ]:
def generate_message(data):
    return f"How many {data} are in the image? Only output an integer between 1 and 5."

def probe(batch):
    # layers should be between 0 (input embedding) and 28 (final activations)
    inputs = processor.process(
        text=generate_message(batch[0]["text"]),
        images=batch[0]["image"],
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

    space_token = torch.tensor([[220]], device=model.device, dtype=inputs["input_ids"].dtype).expand(batch_size, -1)
    inputs["input_ids"] = torch.cat([inputs["input_ids"], space_token], dim=1)

    with torch.no_grad():
        outputs = model(
            **inputs,
            output_hidden_states=True,
            return_dict=True
        )

    # print(len(outputs.hidden_states)) # 29

    # shape of outputs.hidden_states (batch_size, sequence_length, hidden_size)
    # get the predicted output
    last_layer = outputs.hidden_states[-1]
    final_vec_raw = last_layer[:, -1, :]
    final_vec_normed = model.model.transformer.ln_f(final_vec_raw)
    logits_manual = model.model.transformer.ff_out(final_vec_normed)
    token_id = torch.argmax(logits_manual, dim=-1)
    predicted_tokens = [processor.tokenizer.decode(token_id)]
    targets = [item["label"] for item in batch]
    # print(predicted_tokens, targets)

    return predicted_tokens, targets, [hs[:, -1, :] for hs in outputs.hidden_states]

In [ ]:
processor.tokenizer.padding_side = "left"
layers = 29
batch_size = 1
activations = [[[] for _ in range(5)] for _ in range(layers)]
predictions = [[] for _ in range(5)]

for i in range(0, len(dataset), batch_size):
    batch = dataset[i:i+batch_size]
    predicted_tokens, targets, batch_acts = probe(batch)
    for j in range(batch_size):
        predictions[targets[j]-1].append(predicted_tokens[j])
    for j in range(layers):
        for k in range(batch_size):
            label = batch[k]["label"] - 1
            activations[j][label].append(batch_acts[j][k].float().cpu().numpy())
    print(f"batch {i//batch_size+1}/{len(dataset)//batch_size}")


In [ ]:
# accuracies
for i in range(5):
    cnt = 0
    for j in range(len(predictions[i])):
        if predictions[i][j] == str(i+1):
            cnt += 1
    print(f"{i+1}: {cnt/len(predictions[i])}")

In [ ]:
output_folder = "molmo_plots"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i in range(layers):
    all_acts = []
    for x in activations[i]:
        all_acts.extend(x)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(all_acts)
    plt.figure(figsize=(10, 6))

    for j in range(5):
        curr = X_pca[j*group_size:(j+1)*group_size]
        plt.scatter(curr[:, 0], curr[:, 1], label=f"{j+1}")

    plt.title(f"Layer {i}")
    plt.legend()
    save_path = os.path.join(output_folder, f"molmo_{i}.png")
    plt.savefig(save_path)
    plt.show()
    plt.close()

In [ ]:
import shutil
from google.colab import files

shutil.make_archive(output_folder, 'zip', output_folder)
files.download(f'{output_folder}.zip')